In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import Input Data

# 1. S&P 500
sp500 = pd.read_csv('sp500.csv', parse_dates=['DateTime'])

# 2. Inflation
inflation = pd.read_csv('inflation.csv', parse_dates=['DateTime'])

In [3]:
def preprocess_to_daily(df, date_col, value_col):
    # 1.align the datetime format
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    # 2. Drop na
    df = df.dropna(subset=[date_col, value_col])
    
    # 3. Set date column as index
    df = df.set_index(date_col)
    
    # 4. Arrenge the earliest date & the lastest date by days
    full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    
    # 5. Rearrenge the date column
    df_daily = df.reindex(full_date_range)
    
    # 6. Re-index
    df_daily.index.name = date_col
    
    # 7. Fill backwards
    df_daily[value_col] = df_daily[value_col].fillna(method='bfill')
    
    # 8. Re-index, formatted as "YYYY-MM-DD"
    df_daily = df_daily.reset_index()
    df_daily[date_col] = df_daily[date_col].dt.strftime('%Y-%m-%d')
    
    return df_daily

In [4]:
# 1. S&P 500
input_sp500 = preprocess_to_daily(sp500, 'DateTime', 'S&P 500')

input_sp500.head()

/var/folders/x1/zr3m031s4753mpj3_g141cxw0000gn/T/ipykernel_77622/1826281989.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_daily[value_col] = df_daily[value_col].fillna(method='bfill')


,DateTime,S&P 500
0,1871-01-01,4.44
1,1871-01-02,4.50
2,1871-01-03,4.50
3,1871-01-04,4.50
4,1871-01-05,4.50


In [5]:
input_sp500.tail()

,DateTime,S&P 500
56180,2024-10-25,5808.12
56181,2024-10-26,5823.52
56182,2024-10-27,5823.52
56183,2024-10-28,5823.52
56184,2024-10-29,5832.92


In [6]:
# 2. Inflation
input_inflation = preprocess_to_daily(inflation, 'DateTime', 'Inflation Rate')

input_inflation.head()

/var/folders/x1/zr3m031s4753mpj3_g141cxw0000gn/T/ipykernel_77622/1826281989.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_daily[value_col] = df_daily[value_col].fillna(method='bfill')


,DateTime,Inflation Rate
0,1872-01-01,1.527038
1,1872-01-02,2.255462
2,1872-01-03,2.255462
3,1872-01-04,2.255462
4,1872-01-05,2.255462


In [7]:
input_inflation.tail()

,DateTime,Inflation Rate
55757,2024-08-28,2.440633
55758,2024-08-29,2.440633
55759,2024-08-30,2.440633
55760,2024-08-31,2.440633
55761,2024-09-01,2.440633


In [8]:
# Set DateTime as datetime format
input_sp500['DateTime'] = pd.to_datetime(input_sp500['DateTime'])
input_inflation['DateTime'] = pd.to_datetime(input_inflation['DateTime'])

# Re-index
input_sp500 = input_sp500.set_index('DateTime')
input_inflation = input_inflation.set_index('DateTime')

# Turn into monthly data (use month start)
input_sp500_monthly = input_sp500.resample('MS').first()  # 'MS' is for Month Start
input_inflation_monthly = input_inflation.resample('MS').first()

# Display the first few rows
input_sp500_monthly.head()

,S&P 500
DateTime,
1871-01-01,4.44
1871-02-01,4.50
1871-03-01,4.61
1871-04-01,4.74
1871-05-01,4.86


In [9]:
input_inflation_monthly.head()

,Inflation Rate
DateTime,
1872-01-01,1.527038
1872-02-01,2.255462
1872-03-01,2.255462
1872-04-01,2.255462
1872-05-01,2.255462


In [10]:
# Align start date and end date
start_date = max(input_sp500_monthly.index.min(), input_inflation_monthly.index.min())
end_date = min(input_sp500_monthly.index.max(), input_inflation_monthly.index.max())

# Align
input_sp500_monthly = input_sp500_monthly.loc[start_date:end_date]
input_inflation_monthly = input_inflation_monthly.loc[start_date:end_date]

# Merge two datasets
aligned_data = pd.merge(input_sp500_monthly, input_inflation_monthly, left_index=True, right_index=True)

aligned_data.head()

,S&P 500,Inflation Rate
DateTime,,
1872-01-01,4.86,1.527038
1872-02-01,4.88,2.255462
1872-03-01,5.04,2.255462
1872-04-01,5.18,2.255462
1872-05-01,5.18,2.255462


In [11]:
aligned_data.tail()

,S&P 500,Inflation Rate
DateTime,,
2024-05-01,5018.39,3.269029
2024-06-01,5283.40,2.971397
2024-07-01,5475.09,2.894753
2024-08-01,5446.68,2.530730
2024-09-01,5528.93,2.440633


In [12]:
# Apply SMA (Simple Moving Average) with a window size of 12
aligned_data['SP500_sma'] = aligned_data['S&P 500'].rolling(window=2).mean()
aligned_data['Inflation_sma'] = aligned_data['Inflation Rate'].rolling(window=2).mean()

# Calculate first-derivative for SMA
aligned_data['SP500_derivative'] = aligned_data['SP500_sma'].diff()
aligned_data['Inflation_derivative'] = aligned_data['Inflation_sma'].diff()

# SP500 is growth; Inflation is inflation
aligned_data['Growth'] = aligned_data['SP500_derivative']
aligned_data['Inflation'] = aligned_data['Inflation_derivative']

# Define four regimes based on Growth and Inflation
def classify_regime(row):
    if row['Growth'] > 0 and row['Inflation'] > 0:
        return 1  # Regime 1: High G + High I
    elif row['Growth'] > 0 and row['Inflation'] <= 0:
        return 2  # Regime 2: High G + Low I
    elif row['Growth'] <= 0 and row['Inflation'] > 0:
        return 3  # Regime 3: Low G + High I
    else:
        return 4  # Regime 4: Low G + Low I

aligned_data['Regime'] = aligned_data.apply(classify_regime, axis=1)

# Display the first few rows of Growth, Inflation, and Regime
aligned_data[['Growth', 'Inflation', 'Regime']].head()

,Growth,Inflation,Regime
DateTime,,,
1872-01-01,NaN,NaN,4
1872-02-01,NaN,NaN,4
1872-03-01,0.09,0.364212,1
1872-04-01,0.15,0.000000,2
1872-05-01,0.07,0.000000,2


In [13]:
aligned_data[['Growth', 'Inflation', 'Regime']].tail()

,Growth,Inflation,Regime
DateTime,,,
2024-05-01,-59.345,-0.104178,4
2024-06-01,19.815,-0.192983,2
2024-07-01,228.350,-0.187138,2
2024-08-01,81.640,-0.220333,2
2024-09-01,26.920,-0.227060,2


In [14]:
aligned_data.head()

,S&P 500,Inflation Rate,SP500_sma,Inflation_sma,SP500_derivative,Inflation_derivative,Growth,Inflation,Regime
DateTime,,,,,,,,,
1872-01-01,4.86,1.527038,NaN,NaN,NaN,NaN,NaN,NaN,4
1872-02-01,4.88,2.255462,4.87,1.891250,NaN,NaN,NaN,NaN,4
1872-03-01,5.04,2.255462,4.96,2.255462,0.09,0.364212,0.09,0.364212,1
1872-04-01,5.18,2.255462,5.11,2.255462,0.15,0.000000,0.15,0.000000,2
1872-05-01,5.18,2.255462,5.18,2.255462,0.07,0.000000,0.07,0.000000,2


In [15]:
output_data = aligned_data[['S&P 500', 'Inflation Rate', 'Growth', 'Inflation', 'Regime']]

output_file_path = '/Users/kelseyshi/Downloads/week9/input_data.csv'
output_data.to_csv(output_file_path, index=True)